In [13]:
# Importações necessárias
import pandas as pd
import re
import nltk
import string 
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
import datetime as dt
import spacy
#from bertopic import BERTopic
#from sklearn.feature_extraction.text import CountVectorizer
from groq import Groq
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
import datetime
import os
from textwrap import wrap
import time
import plotly.io as pio

In [3]:
nltk.download('stopwords')
stop_words_pt = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\familia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Carregar o modelo em português
nlp = spacy.load("pt_core_news_sm")

In [7]:
# Carregar o DataFrame a partir do arquivo Excel
df_final = pd.read_excel('Base_dados_Telegram.xlsx')

In [15]:
# 1. Gráfico interativo de quantidade de mensagens por data
def grafico_mensagens_por_data(df):
    mensagens_por_data = df.groupby(df['data'].dt.date).size()
    mensagens_por_data_df = mensagens_por_data.reset_index(name='Quantidade de Mensagens')
    mensagens_por_data_df['data'] = mensagens_por_data_df['data'].apply(lambda x: x.strftime('%d-%m-%Y'))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mensagens_por_data_df['data'],
                             y=mensagens_por_data_df['Quantidade de Mensagens'],
                             mode='lines',
                             name='Quantidade de Mensagens'))

    fig.update_layout(
        xaxis=dict(
            title='Data (Dia-Mês-Ano)',
            range=[mensagens_por_data_df['data'].iloc[0], mensagens_por_data_df['data'].iloc[29]],
            rangeslider=dict(
                visible=True,
                range=[mensagens_por_data_df['data'].iloc[0], mensagens_por_data_df['data'].iloc[-1]]
            ),
            tickformat='%d-%m-%Y',
        ),
        yaxis=dict(title='Quantidade de Mensagens'),
        title='Quantidade de Mensagens por Data',
        dragmode='zoom',
        hovermode='x unified'
    )
    pio.renderers.default = "browser" 
    fig.show()

In [16]:
grafico_mensagens_por_data(df_final)

In [19]:
# 2. Gráfico interativo de quantidade de mensagens por autor e data
def grafico_mensagens_por_autor(df, autores_especificos):
    mensagens_por_autor_data_df = df.groupby(['data', 'username']).size().reset_index(name='Quantidade de Mensagens')
    filtro_df = mensagens_por_autor_data_df[mensagens_por_autor_data_df['username'].isin(autores_especificos)]

    fig1 = px.line(filtro_df,
                   x='data',
                   y='Quantidade de Mensagens',
                   color='username',
                   labels={"Data": "Data (Dia-Mês-Ano)", "Quantidade de Mensagens": "Quantidade de Mensagens"},
                   title='Quantidade de Mensagens por Autor e Data')

    fig1.update_layout(
        xaxis=dict(
            title='Data (Dia-Mês-Ano)',
            range=[filtro_df['data'].iloc[0], filtro_df['data'].iloc[29]],
            rangeslider=dict(
                visible=True,
                range=[filtro_df['data'].iloc[0], filtro_df['data'].iloc[-1]]
            ),
            tickformat='%d-%m-%Y',
        ),
        yaxis=dict(title='Quantidade de Mensagens'),
        dragmode='zoom',
        hovermode='x unified'
    )
    pio.renderers.default = "browser" 
    fig1.show()

In [18]:
grafico_mensagens_por_autor(df_final, ['jairbolsonarobrasil', 'LulanoTelegram'])

In [20]:
# 3. Gráficos de barras para emojis por grupo
def criar_grafico_reacoes(df, username):
    df['emojis_list'] = df['emojis'].str.split(', ')
    df['counts_list'] = df['counts'].str.split(', ').apply(lambda x: list(map(int, x)))
    df_expanded = df.explode(['emojis_list', 'counts_list'])
    df_expanded = df_expanded.rename(columns={'emojis_list': 'emoji', 'counts_list': 'count'})
    
    df_user = df_expanded[df_expanded['username'] == username]
    df_grouped = df_user.groupby('emoji', as_index=False)['count'].sum()

    fig = px.bar(df_grouped,
                 x='emoji',
                 y='count',
                 labels={'emoji': 'Reação (Emoji)', 'count': 'Quantidade de Reações'},
                 title=f'Quantidade de Reações por Emoji - {username}')
    pio.renderers.default = "browser" 
    fig.show()


In [22]:
criar_grafico_reacoes(df_final, 'jairbolsonarobrasil')

In [23]:
criar_grafico_reacoes(df_final, 'LulanoTelegram')

In [24]:
# Filtrar os usernames de cada grupo
bolsonaro_usernames = ["JNA2021", "filipetrielli1", "lulajamais2", "ianmaldonado", "realpfigueiredo",
                       "allandossantos", "siteolavete", "DireitaChannel", "FlaviaFerronato", "FabianaBarroso",
                       "aazzibarreto", "isentaocast", "eduperez80", "ludmilalinsgrilo", "JulioVSchneider",
                       "conexaopoliticabr", "PHVox", "arthurweintraub", "diretoaosfatos", "opropriolavo",
                       "domesdras", "filgmartin", "vidadestra", "grupob38", "souzaslaughter"]

lula_usernames = ["jornalistaslivres", "esquerdaonline", "progressistass", "+VPXlha9FZOcxOTgx", "voulula13",
                  "socialismos", "memesesquerdistas", "esquerdasunidascanal", "Andrejanonestelegram",
                  "lulaverso", "arsenaldolula"]

neutros_usernames = ["avozdocerrado", "garoaclube", "bbcbrasil", "tramadora", "SputnikBrasil"]

In [25]:
# Criar DataFrames filtrados por grupo
df_bolsonaro = df_final[df_final['username'].isin(bolsonaro_usernames)]
df_lula = df_final[df_final['username'].isin(lula_usernames)]
df_neutros = df_final[df_final['username'].isin(neutros_usernames)]

In [26]:
# Agrupar por data e contar a quantidade de mensagens
bolsonaro_por_data = df_bolsonaro.groupby('data').size().reset_index(name='Quantidade de Mensagens')
lula_por_data = df_lula.groupby('data').size().reset_index(name='Quantidade de Mensagens')
neutros_por_data = df_neutros.groupby('data').size().reset_index(name='Quantidade de Mensagens')

In [27]:
# Adicionar coluna para identificar o grupo
bolsonaro_por_data['Grupo'] = 'Bolsonaro'
lula_por_data['Grupo'] = 'Lula'
neutros_por_data['Grupo'] = 'Neutros'

In [28]:
# Concatenar os DataFrames
df_mensagens_por_data = pd.concat([bolsonaro_por_data, lula_por_data, neutros_por_data])

In [29]:
# Criar o gráfico de linhas
fig2 = px.line(df_mensagens_por_data,
              x='data',
              y='Quantidade de Mensagens',
              color='Grupo',
              labels={
                  "data": "Data",
                  "Quantidade de Mensagens": "Quantidade de Mensagens"
              },
              title='Quantidade de Mensagens por Data (Perfis Não Institucionais)')

# Ajustar a exibição inicial e adicionar funcionalidades de zoom e rolagem
fig2.update_layout(
    xaxis=dict(
        title='Data (Dia-Mês-Ano)',
        rangeslider=dict(
            visible=True,
            range=[df_mensagens_por_data['data'].min(), df_mensagens_por_data['data'].max()]  # Habilitar rolagem para o restante do período
        ),
        tickformat='%d-%m-%Y',  # Formato da data
    ),
    yaxis=dict(
        title='Quantidade de Mensagens'
    ),
    dragmode='zoom',  # Habilitar o modo de zoom
    hovermode='x unified'  # Exibir a informação da quantidade de mensagens ao passar o mouse
)

# Exibir o gráfico
fig2.show()

In [30]:
# Gráfico de Reações por Emoji para Perfis Não Institucionais

# Função para criar gráficos de barras de reações para cada grupo
def criar_grafico_reacoes_grupo(df, grupo_usernames, grupo_nome):
    df_grupo = df[df['username'].isin(grupo_usernames)]
    df_grouped = df_grupo.groupby('emoji', as_index=False)['count'].sum()

    fig = px.bar(df_grouped,
                 x='emoji',
                 y='count',
                 labels={
                     'emoji': 'Reação (Emoji)',
                     'count': 'Quantidade de Reações'
                 },
                 title=f'Quantidade de Reações por Emoji - {grupo_nome}')
    return fig

In [31]:
# Separar os emojis e as contagens em listas
df_final['emojis_list'] = df_final['emojis'].str.split(', ')
df_final['counts_list'] = df_final['counts'].str.split(', ').apply(lambda x: list(map(int, x)))


In [32]:
# Expandir as listas em várias linhas
df_expanded = df_final.explode(['emojis_list', 'counts_list'])
# Renomear as colunas expandidas
df_expanded = df_expanded.rename(columns={'emojis_list': 'emoji', 'counts_list': 'count'})

In [33]:
# Criar gráficos para os grupos
fig_reacoes_bolsonaro = criar_grafico_reacoes_grupo(df_expanded, bolsonaro_usernames, 'Bolsonaro (Perfis Não Institucionais)')
fig_reacoes_bolsonaro.show()

In [34]:
fig_reacoes_lula = criar_grafico_reacoes_grupo(df_expanded, lula_usernames, 'Lula (Perfis Não Institucionais)')
fig_reacoes_lula.show()


In [35]:
fig_reacoes_neutros = criar_grafico_reacoes_grupo(df_expanded, neutros_usernames, 'Neutros (Perfis Não Institucionais)')
fig_reacoes_neutros.show()

In [36]:
def limpar_mensagem(mensagem):
    if isinstance(mensagem, str):  # Verifica se a mensagem é uma string
        # Converter para minúsculas
        mensagem = mensagem.lower()

        # Remover números
        mensagem = re.sub(r'\d+', '', mensagem)

        # Remover URLs completamente
        mensagem = re.sub(r'http\S+|www\S+|https\S+', '', mensagem, flags=re.MULTILINE)

        # Remover espaços extras
        mensagem = mensagem.strip()
        mensagem = re.sub(r'\s+', ' ', mensagem)

        # Remover stop words
        palavras = mensagem.split()
        mensagem = ' '.join([palavra for palavra in palavras if palavra not in stop_words_pt])

        return mensagem
    else:
        return ''  # Retorna uma string vazia se não for uma string

In [37]:
# Aplicar a função de limpeza na coluna de conteúdo
df_final['Conteúdo Limpo'] = df_final['conteudo'].apply(limpar_mensagem)

In [38]:
# Agrupar o conteúdo por data e concatenar o conteúdo limpo
conteudo_por_data = df_final.groupby('data')['Conteúdo Limpo'].apply(' '.join).reset_index()

In [39]:
# Remover registros sem conteúdo e agrupar por dia
conteudo_com_texto = conteudo_por_data[conteudo_por_data['Conteúdo Limpo'].str.strip() != '']
conteudo_com_texto['Dia'] = conteudo_com_texto['data'].dt.date
conteudo_agrupado = conteudo_com_texto.groupby('Dia')['Conteúdo Limpo'].apply(' '.join).reset_index()

C:\Users\familia\AppData\Local\Temp\ipykernel_3848\3543405658.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
# Função para gerar nuvem de palavras
def gerar_nuvem_palavras(texto, titulo="Nuvem de Palavras"):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(titulo, fontsize=16)
    plt.show()

In [41]:
# Criar DataFrames para cada grupo específico
df_jairbolsonarobrasil = df_final[df_final['username'] == 'jairbolsonarobrasil']
df_lulanoTelegram = df_final[df_final['username'] == 'LulanoTelegram']
df_bolsonaro_usernames = df_final[df_final['username'].isin(bolsonaro_usernames)]
df_lula_usernames = df_final[df_final['username'].isin(lula_usernames)]
df_neutros_usernames = df_final[df_final['username'].isin(neutros_usernames)]

In [42]:
# Filtrar e agrupar o conteúdo limpo por data para cada grupo
conteudo_jairbolsonarobrasil_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_jairbolsonarobrasil['data'].dt.date)]
conteudo_lulanoTelegram_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_lulanoTelegram['data'].dt.date)]
conteudo_bolsonaro_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_bolsonaro_usernames['data'].dt.date)]
conteudo_lula_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_lula_usernames['data'].dt.date)]
conteudo_neutros_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_neutros_usernames['data'].dt.date)]


In [43]:
# Função para gerar nuvem de palavras
def gerar_nuvem_palavras(texto):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [44]:
# Converter a coluna 'Dia' para datetime, se necessário
if not pd.api.types.is_datetime64_any_dtype(conteudo_agrupado['Dia']):
    conteudo_agrupado['Dia'] = pd.to_datetime(conteudo_agrupado['Dia'])

In [45]:
# Criar um dropdown interativo com todas as datas disponíveis
datas_disponiveis = conteudo_agrupado['Dia'].dt.strftime('%d-%m-%Y').unique()
dropdown = widgets.Dropdown(
    options=datas_disponiveis,
    description='data:',
    disabled=False,
)

In [46]:
# Função para mostrar a nuvem de palavras para o dia selecionado em diferentes grupos
def mostrar_nuvem_por_grupo(data_selecionada, grupo):
    data_formato = pd.to_datetime(data_selecionada, format='%d-%m-%Y').date()

    if grupo == 'jairbolsonarobrasil':
        filtro_data = conteudo_jairbolsonarobrasil_agrupado[conteudo_jairbolsonarobrasil_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'lulanoTelegram':
        filtro_data = conteudo_lulanoTelegram_agrupado[conteudo_lulanoTelegram_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'bolsonaro_usernames':
        filtro_data = conteudo_bolsonaro_usernames_agrupado[conteudo_bolsonaro_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'lula_usernames':
        filtro_data = conteudo_lula_usernames_agrupado[conteudo_lula_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'neutros_usernames':
        filtro_data = conteudo_neutros_usernames_agrupado[conteudo_neutros_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']

    if not filtro_data.empty:
        texto = filtro_data.values[0]
        gerar_nuvem_palavras(texto)
    else:
        print(f"Nenhum conteúdo encontrado para a data: {data_selecionada}")

In [49]:
# Conectar a função de exibição da nuvem de palavras ao dropdown
interact(mostrar_nuvem_por_grupo, data_selecionada=dropdown, grupo=['jairbolsonarobrasil', 'lulanoTelegram', 'bolsonaro_usernames', 'lula_usernames', 'neutros_usernames'])

interactive(children=(Dropdown(description='data:', options=('17-12-2015', '18-12-2015', '19-12-2015', '21-12-…

<function __main__.mostrar_nuvem_por_grupo(data_selecionada, grupo)>

In [36]:
#Filtrar os usernames de cada grupo
bolsonaro_usernames1 = ["JNA2021", "filipetrielli1", "lulajamais2", "ianmaldonado", "realpfigueiredo",
                       "allandossantos", "siteolavete", "DireitaChannel", "FlaviaFerronato", "FabianaBarroso",
                       "aazzibarreto", "isentaocast", "eduperez80", "ludmilalinsgrilo", "JulioVSchneider",
                       "conexaopoliticabr", "PHVox", "arthurweintraub", "diretoaosfatos", "opropriolavo",
                       "domesdras", "filgmartin", "vidadestra", "grupob38", "souzaslaughter", "jairbolsonarobrasil"]

lula_usernames1 = ["jornalistaslivres", "esquerdaonline", "progressistass", "+VPXlha9FZOcxOTgx", "voulula13",
                  "socialismos", "memesesquerdistas", "esquerdasunidascanal", "Andrejanonestelegram",
                  "lulaverso", "arsenaldolula", "LulanoTelegram"]

neutros_usernames1 = ["avozdocerrado", "garoaclube", "bbcbrasil", "tramadora", "SputnikBrasil"]

In [37]:
# Criar DataFrames filtrados por grupo
df_bolsonaro1 = df_final[df_final['username'].isin(bolsonaro_usernames1)]
df_lula1 = df_final[df_final['username'].isin(lula_usernames1)]
df_neutros1 = df_final[df_final['username'].isin(neutros_usernames1)]

In [38]:
# Função para lematizar o texto
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

In [39]:
# Supondo que você tenha uma coluna 'mensagem' com os textos
df_bolsonaro1['mensagem_lemmatizada'] = df_bolsonaro1['Conteúdo Limpo'].apply(lemmatize_text)

In [40]:
df_lula1['mensagem_lemmatizada'] = df_lula1['Conteúdo Limpo'].apply(lemmatize_text)
df_neutros1['mensagem_lemmatizada'] = df_neutros1['Conteúdo Limpo'].apply(lemmatize_text)

In [41]:
# Extrair as mensagens lematizadas
documents = df_bolsonaro1['mensagem_lemmatizada'].tolist()
documents1= df_lula1['mensagem_lemmatizada'].tolist()
documents2= df_neutros1['mensagem_lemmatizada'].tolist()

In [42]:
# Customizar o CountVectorizer
vectorizer_model = CountVectorizer(
    stop_words=stop_words_pt,  # Usar stopwords em português do NLTK
    ngram_range=(1, 2),  # Considerar uni e bigramas
    min_df=5,  # Ignorar termos que aparecem em menos de 5 documentos
    max_df=0.95  # Ignorar termos que aparecem em mais de 95% dos documentos
)


In [43]:
# Instanciar o modelo BERTopic com o vetor customizado
topic_model = BERTopic(vectorizer_model=vectorizer_model)

In [44]:
topics, _ = topic_model.fit_transform(documents)
# Atribuir os tópicos às mensagens Bolsonaristas
df_bolsonaro1['topic'] = topics


In [45]:
topics1, _ = topic_model.fit_transform(documents1)
# Atribuir os tópicos às mensagens Lula
df_lula1['topic'] = topics1

In [46]:
topics2, _ = topic_model.fit_transform(documents2)
# Atribuir os tópicos às mensagens Neutros
df_neutros1['topic'] = topics2

In [2]:
# Carregar o DataFrame a partir do arquivo Excel
df_bolsonaro1 = pd.read_excel('base_bolsonaro.xlsx')

In [3]:
client = Groq(
    api_key="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"
)

In [4]:
resumos_por_data_usuario = {}  # Dicionário global para armazenar resumos por data e usuário

def visualizar_resumo_por_topico(df_selecionado, topico, limpar_resumos=True):
    global resumos_por_data_usuario  # Usar a variável global

    if limpar_resumos:
        resumos_por_data_usuario = {}  # Limpar resumos anteriores

    # Filtrar o dataframe pelo tópico selecionado
    df_topico = df_selecionado[df_selecionado['topic'] == topico]

    # Agrupar as mensagens por data e usuário
    grupo_por_data_usuario = df_topico.groupby(['data', 'username'])

    print(f"Resumo do Tópico {topico} agrupado por data e usuário:")

    # Iterar pelas combinações de data e usuário e gerar um resumo das mensagens de cada uma
    for (data, usuario), grupo in grupo_por_data_usuario:
        mensagens = grupo['conteudo'].dropna().tolist()  # Remove NaNs e converte para lista
        mensagens = [str(mensagem) for mensagem in mensagens if isinstance(mensagem, str)]  # Filtra apenas strings

        if mensagens:  # Verifica se a lista de mensagens não está vazia
            mensagens_texto = "\n".join(mensagens)  # Concatena as mensagens para enviar ao modelo

            # Enviar as mensagens ao modelo Llama para obter um resumo
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": f"Abaixo estão algumas mensagens de {data} do usuário {usuario} sobre o Tópico {topico}. Por favor, resuma o principal assunto discutido nessas mensagens:\n\n{mensagens_texto}"
                    }
                ],
                model="llama3-70b-8192"
            )

            # Extrair o conteúdo da resposta do modelo
            resumo = chat_completion.choices[0].message.content
            resumos_por_data_usuario[(data, usuario)] = resumo  # Armazenar o resumo no dicionário

            print(f"\nData: {data} | Usuário: {usuario}")
            print(f"Resumo: {resumo}")
        else:
            print(f"\nData: {data} | Usuário: {usuario} - Sem mensagens disponíveis para resumo.")

    return resumos_por_data_usuario

In [5]:
# Função para salvar o resumo em PDF na pasta especificada
def salvar_resumo_pdf1(resumos, topico):
    # Define a pasta onde os arquivos PDF serão salvos
    pasta_destino = "Relatorio de mensagens dos perfies bolsonaro"
    os.makedirs(pasta_destino, exist_ok=True)  # Cria a pasta se ela não existir

    # Define o nome do arquivo PDF com base no tópico
    nome_arquivo = os.path.join(pasta_destino, f"resumo_topico_{topico}.pdf")

    # Configura o PDF com tamanho A4
    c = canvas.Canvas(nome_arquivo, pagesize=A4)
    largura, altura = A4

    # Define a posição inicial para o texto
    x, y = 50, altura - 50
    c.setFont("Helvetica", 10)

    # Adiciona título no PDF
    c.drawString(x, y, f"Resumo do Tópico {topico}")
    y -= 30

    # Define a largura máxima da linha em caracteres
    largura_maxima_linha = 80  # Ajuste conforme necessário para o layout desejado

    # Itera pelas datas e resumos para salvá-los no PDF
    for data, resumo in resumos.items():
        # Adiciona a data ao PDF
        c.drawString(x, y, f"Data: {data}")
        y -= 15

        # Quebra o texto do resumo em várias linhas, se necessário
        linhas_resumo = wrap(resumo, largura_maxima_linha)
        for linha in linhas_resumo:
            if y < 40:  # Verifica se há espaço suficiente, caso contrário, cria nova página
                c.showPage()
                c.setFont("Helvetica", 10)
                y = altura - 50
            c.drawString(x, y, linha)
            y -= 15  # Move para a linha seguinte

        y -= 10  # Adiciona um espaço entre resumos

    # Salva o PDF
    c.save()
    print(f"O PDF foi salvo em: {nome_arquivo}")

In [6]:
# Loop para gerar resumos para tópicos de 1 a 42 e salvar em PDF
for topico in range(808, 958):
    resumos = visualizar_resumo_por_topico(df_bolsonaro1, topico, limpar_resumos=True)
    salvar_resumo_pdf1(resumos, topico)
     #Aguarde alguns segundos antes de continuar (ajuste o valor conforme necessário)
    time.sleep(3)

Resumo do Tópico 808 agrupado por data e usuário:

Data: 2020-09-15 00:52:47 | Usuário: aazzibarreto
Resumo: O principal assunto discutido nessas mensagens é o compartilhamento de um link de vídeo para uma palestra do usuário aazzibarreto na FUNAG.

Data: 2021-03-26 04:05:46 | Usuário: FabianaBarroso
Resumo: O principal assunto discutido nessas mensagens é a conversão de C.S. Lewis, autor de "As Crônicas de Nárnia", do ateísmo ao cristianismo, após conversas com seu amigo J.R.R. Tolkien e ler o livro "The Everlasting Man" de G.K. Chesterton, e como ele posteriormente ministrou palestras sobre os fundamentos da fé cristã na BBC durante a II Guerra Mundial.

Data: 2021-08-25 15:20:02 | Usuário: FabianaBarroso
Resumo: O principal assunto discutido nas mensagens é sobre uma palestra realizada no Conacon, em Ribeirão Preto, no dia anterior (24/08/2021).

Data: 2021-09-10 22:02:04 | Usuário: PHVox
Resumo: O principal assunto discutido nessas mensagens é a divulgação de um evento (palestra) s

In [55]:
df_neutros1.to_excel('base_neutros.xlsx', index=False)

In [ ]:
# abaixo estou apenas usando o codigo 

In [3]:
import emoji  

In [4]:
dflula = pd.read_csv("C:\\Users\\familia\\OneDrive\\Área de Trabalho\\competição\\arquivolula.csv")
dfbolsonaro = pd.read_csv("C:\\Users\\familia\\OneDrive\\Área de Trabalho\\competição\\arquivobolsonaro.csv")

In [7]:
def remover_emojis(texto):
    # Expressão regular para capturar emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
        "\U0001F680-\U0001F6FF"  # Transporte e símbolos relacionados
        "\U0001F1E0-\U0001F1FF"  # Bandeiras (códigos de região)
        "\U00002700-\U000027BF"  # Diversos símbolos
        "\U000024C2-\U0001F251"  # Símbolos adicionais
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', texto)

In [10]:
def limpar_mensagem1(mensagem):
    if isinstance(mensagem, str):  # Verifica se a mensagem é uma string
        # Converter para minúsculas
        mensagem = mensagem.lower()

        # Remover números
        mensagem = re.sub(r'\d+', '', mensagem)

        # Remover URLs completamente
        mensagem = re.sub(r'http\S+|www\S+|https\S+', '', mensagem, flags=re.MULTILINE)

        # Remover emojis
        mensagem = remover_emojis(mensagem)

        # Remover espaços extras
        mensagem = mensagem.strip()
        mensagem = re.sub(r'\s+', ' ', mensagem)

        # Remover stop words
        palavras = mensagem.split()
        mensagem = ' '.join([palavra for palavra in palavras if palavra not in stop_words_pt])

        return mensagem
    else:
        return ''  # Retorna uma string vazia se não for uma string

In [11]:
# Limpar os textos dos datasets
dflula['texto_comentario_limpo'] = dflula['texto_comentario'].apply(limpar_mensagem1)
dfbolsonaro['texto_comentario_limpo'] = dfbolsonaro['texto_comentario'].apply(limpar_mensagem1)

In [14]:
#data_modificacao_comentarios

In [12]:
# Perfis únicos de cada base
lula_perfis = set(dflula['username_comentario'].unique())
bolsonaro_perfis = set(dfbolsonaro['username_comentario'].unique())
# Perfis presentes em ambas as bases
interseccao = lula_perfis & bolsonaro_perfis
# Perfis exclusivos de Lula
so_lula = lula_perfis - bolsonaro_perfis
# Perfis exclusivos de Bolsonaro
so_bolsonaro = bolsonaro_perfis - lula_perfis

print('interseccao',len(interseccao))
print('lula',len(so_lula))
print('bolsonaro',len(so_bolsonaro))

interseccao 132539
lula 1186085
bolsonaro 2071150


In [16]:
# Adiciona uma coluna indicando o grupo de cada comentário
def classificar_grupo(username):
    if username in interseccao:
        return "interseccao"
    elif username in so_lula:
        return "so_lula"
    elif username in so_bolsonaro:
        return "so_bolsonaro"
    else:
        return "indefinido"


In [17]:
# Aplica a classificação de grupo aos dataframes
dflula['grupo'] = dflula['username_comentario'].apply(classificar_grupo)
dfbolsonaro['grupo'] = dfbolsonaro['username_comentario'].apply(classificar_grupo)


In [18]:
# Combina os dois dataframes em um único
df_combined = pd.concat([dflula, dfbolsonaro], ignore_index=True)

# Agrupa os dados por grupo e data
df_agrupado = df_combined.groupby(['grupo', 'data_modificacao_comentarios'])['texto_comentario_limpo'].apply(' '.join).reset_index()

In [19]:
# Salva o dataframe agrupado como CSV
df_agrupado.to_csv('comentarios_agrupados_por_grupo_data.csv', index=False)